In [1]:
from matplotlib import pyplot as plt
import numpy as np
from astropy.table import Table
import os,shutil,glob,sys,traceback
from astropy.cosmology import LambdaCDM
import pdb
import pickle
from astropy.io import fits
from astroquery.mast import Observations
import jhat
from jhat import jwst_photclass,hst_photclass,st_wcs_align
import space_phot
import astropy
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
from jwst.pipeline import Image3Pipeline
import pprint
from astroquery.mast import Observations
import subprocess
import sewpy
from astropy.wcs import WCS

/opt/homebrew/Caskroom/miniconda/base/envs/jwst_ss/lib/python3.11/site-packages/stsci/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)




The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    


<img style="float: center;" src='jWST-SS-1300x300_Banner2.jpg' width="1000px"/> 

# Summer School Week 2
## Pipeline Introduction 

### Author: David Coulter (JHU/STScI), Justin Pierel (STScI), Mike Engesser (STScI)

**Purpose**:<BR>
This notebook will cover the basic concepts of difference imaging for the purpose of discovering transients, and to enable photometry measurements on the resulting sources.

**Data**:<BR>
This example is set up to use an example dataset from the JADES program....
Mention also the instrument with maybe some lonks to JDox

**JWST pipeline version and CRDS context**:<BR>
XXX

<hr style="border:1px solid gray"> </hr>

# Section 1: Building Aligned Lvl3 Mosaics And Projecting Them to the Same Tangential Plane

### Inputs: 
* JHAT-aligned Lvl 2 Cals for each epoch
   * F200W, F277W cals
* At least two epochs
   * Reference epoch, Science epoch

### Outputs:
* Reference and Science mosaics on the same tangential plane

### Steps

1. Visualize the JADES lvl2 images
2. Plot the lvl2 image obs dates as histograms
3. Choose which lvl2s make each epoch (Ref and Sci)
4. For the Ref epoch, create the ASN file and invoke JWST lvl 3 pipeline to create image.
5. Create the Sci epoch:
    1. Use the Ref epoch as input into the Sci epoch drizzle call
    2. Alternatively, create the Sci epoch like the Ref epoch, and use `reproject` to get both images on the same plane.

In [54]:
JADES_PID = '1180'

# Get supernova positions
# https://www.wis-tns.org/object/2023adta
SN2023adta = SkyCoord(53.135283, -27.814517, unit=(u.deg, u.deg), frame='icrs')

# https://www.wis-tns.org/object/2023adsy
SN2023adsy = SkyCoord(53.134853, -27.820899, unit=(u.deg, u.deg), frame='icrs')

# ra=53.13527374999999
# dec=-27.814510305555558
rad=3.0/60.0 # 3 arcmin in deg

obs_table = Observations.query_criteria(
                                        coordinates=f'{SN2023adta.ra.degree} {SN2023adta.dec.degree}',
                                        radius=f"{rad} deg",
                                        instrument_name=['NIRCAM','NIRCAM/IMAGE'],
                                        proposal_id = [JADES_PID],
                                        calib_level=["3"],
                                        filters=['F200W','F356W']
                                        )
obs_table

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID,objID1,distance
str7,str4,str7,str12,str4,str5,str8,str29,str33,str36,float64,float64,str5,str21,int64,float64,float64,float64,float64,float64,str62,float64,str4,str3,int64,str115,str62,str63,str6,bool,float64,str9,str9,str9,float64
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F356W,INFRARED,POINTINGTWO-B,Unidentified; High Latitude Field,jw01180-o010_t009_nircam_clear-f356w,53.18674083333333,-27.786844444444455,image,"Eisenstein, Daniel J.",3,59852.73194975695,59852.88330929398,12368.763000000003,3140.0,3980.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60100.7916666,1180,GTO,--,POLYGON 53.182299114 -27.833338352 53.145083326 -27.81551202 53.191133281 -27.740280887 53.228330869 -27.758094888,mast:JWST/product/jw01180-o010_t009_nircam_clear-f356w_i2d.jpg,mast:JWST/product/jw01180-o010_t009_nircam_clear-f356w_i2d.fits,PUBLIC,False,nan,224248590,811212494,811212494,31.409679248285386
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F356W,INFRARED,POINTINGONE-A,Unidentified; High Latitude Field,jw01180-o009_t008_nircam_clear-f356w,53.175945,-27.800711111111127,image,"Eisenstein, Daniel J.",3,60216.707969965275,60216.86131763889,12368.763000000003,3140.0,3980.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60583.40166657,1180,GTO,--,POLYGON 53.171557867 -27.84758274 53.13381618 -27.82950624 53.180211933 -27.75372245 53.217935016 -27.771786347,mast:JWST/product/jw01180-o009_t008_nircam_clear-f356w_i2d.jpg,mast:JWST/product/jw01180-o009_t008_nircam_clear-f356w_i2d.fits,PUBLIC,False,nan,224200329,810683909,810683909,21.60068379261372
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F200W,INFRARED,GS-MEDIUM-HST,--,jw01180-o028_t028_nircam_clear-f200w,53.15372125,-27.780369444444432,image,"Eisenstein, Daniel J.",3,59860.96615611111,59861.15206190972,5669.0160000000005,1755.0,2226.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60226.34187503,1180,GTO,--,POLYGON 53.08388357 -27.904560956 53.048151945 -27.880367717 53.107414009 -27.811954152 53.143136306 -27.836132126,mast:JWST/product/jw01180-o028_t028_nircam_clear-f200w_i2d.jpg,mast:JWST/product/jw01180-o028_t028_nircam_clear-f200w_i2d.fits,PUBLIC,False,nan,224908641,810475284,810475284,46.60638185210617
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F200W,INFRARED,GS-MEDIUM-HST,--,jw01180-o030_t028_nircam_clear-f200w,53.15372125,-27.780369444444432,image,"Eisenstein, Daniel J.",3,59862.76432083333,59862.80147789352,2834.508,1755.0,2226.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60228.68001146,1180,GTO,--,POLYGON 53.101777837 -27.938477162 53.067470688 -27.915238448 53.126558376 -27.847098384 53.16085661 -27.870322472,mast:JWST/product/jw01180-o030_t028_nircam_clear-f200w_i2d.jpg,mast:JWST/product/jw01180-o030_t028_nircam_clear-f200w_i2d.fits,PUBLIC,False,nan,97276909,810418803,810418803,120.53681652943298
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F356W,INFRARED,MEDS0001-replan,Unidentified; High Latitude Field,jw01180-o219_t219_nircam_clear-f356w,53.16163458333334,-27.822508333333303,image,"Eisenstein, Daniel J.",3,60263.59145759259,60263.66229165509,5669.0160000000005,3140.0,3980.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60631.64543972,1180,GTO,--,POLYGON 53.112599997 -27.846796753 53.109456757 -27.806370889 53.209371694 -27.800258675 53.212551967 -27.840682262,mast:JWST/product/jw01180-o219_t219_nircam_clear-f356w_i2d.jpg,mast:JWST/product/jw01180-o219_t219_nircam_clear-f356w_i2d.fits,PUBLIC,False,nan,192112125,810208411,810208411,0.0
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F356W,INFRARED,POINTINGFOUR-A,Unidentified; High Latitude Field,jw01180-o018_t014_nircam_clear-f356w,53.15209

In [55]:
prod = Observations.get_unique_product_list(obs_table)
prod

INFO: 311499 of 336960 products were duplicates. Only returning 25461 unique product(s). [astroquery.mast.utils]
INFO: To return all products, use `Observations.get_product_list` [astroquery.mast.observations]


obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str6,str4,str51,int64,str9,str6,int64,str5
223810605,JWST,image,jw01180007001_10101_00009_nrcb4,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00011_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00011_asn.json,1803,224007745,PUBLIC,2,F200W
223773483,JWST,image,jw01180007001_10101_00009_nrcb3,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00012_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00012_asn.json,1803,224007745,PUBLIC,2,F200W
223852702,JWST,image,jw01180007001_10101_00009_nrcb2,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00013_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00013_asn.json,1803,224007745,PUBLIC,2,F200W
95382691,JWST,image,jw01180007001_10101_00009_nrcb1,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00014_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00014_asn.json,1803,224007745,PUBLIC,2,F200W
223929256,JWST,image,jw01180007001_10101_00009_nrca4,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00016_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00016_asn.json,1803,224007745,PUBLIC,2,F200W
95382517,JWST,image,jw01180007001_10101_00009_nrca3,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00017_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00017_asn.json,1803,224007745,PUBLIC,2,F200W
223784156,JWST,image,jw01180007001_10101_00009_nrca2,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00018_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00018_asn.json,1803,224007745,PUBLIC,2,F200W
223919306,JWST,image,jw01180007001_10101_00009_nrca1,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00019_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00019_asn.json,1803,224007745,PUBLIC,2,F200W
223882285,JWST,image,jw01180007001_10101_00008_nrcb4,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00021_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00021_asn.json,1803,224007745,PUBLIC,2,F200W


In [56]:
products = Observations.filter_products(prod,
                                        productSubGroupDescription=["CAL"],
                                        extension="fits")
products

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str6,str4,str51,int64,str9,str6,int64,str5
95382616,JWST,image,jw01180007001_06101_00001_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00001_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00001_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W
223922492,JWST,image,jw01180007001_06101_00001_nrcblong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00001_nrcblong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00001_nrcblong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95382563,JWST,image,jw01180007001_06101_00002_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00002_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00002_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95383048,JWST,image,jw01180007001_06101_00002_nrcblong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00002_nrcblong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00002_nrcblong_cal.fits,117573120,224007792,PUBLIC,2,F356W
223881812,JWST,image,jw01180007001_06101_00003_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00003_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00003_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95382947,JWST,image,jw01180007001_06101_00003_nrcblong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00003_nrcblong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00003_nrcblong_cal.fits,117573120,224007792,PUBLIC,2,F356W
223861921,JWST,image,jw01180007001_06101_00004_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00004_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00004_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95383023,JWST,image,jw01180007001_06101_00004_nrcblong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00004_nrcblong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00004_nrcblong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95382620,JWST,image,jw01180007001_06101_00005_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00005_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00005_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W


In [57]:
epoch_1 = ["jw01180015001_10101_00006_nrcb2",
"jw01180015001_10101_00003_nrcb2",
"jw01180015001_10101_00008_nrcb2",
"jw01180015001_10101_00009_nrcb2",
"jw01180015001_10101_00007_nrcb2",
"jw01180015001_10101_00001_nrcb2",
"jw01180015001_10101_00004_nrcb2",
"jw01180015001_10101_00005_nrcb2",
"jw01180015001_10101_00002_nrcb2",
"jw01180015001_06101_00007_nrcblong",
"jw01180015001_06101_00005_nrcblong",
"jw01180015001_06101_00001_nrcblong",
"jw01180015001_06101_00002_nrcblong",
"jw01180015001_06101_00006_nrcblong",
"jw01180015001_06101_00004_nrcblong",
"jw01180015001_06101_00009_nrcblong",
"jw01180015001_06101_00003_nrcblong",
"jw01180015001_06101_00008_nrcblong"]

epoch_2 =[
    "jw01180013001_10101_00009_nrcb2",
    "jw01180013001_10101_00002_nrcb2",
    "jw01180013001_10101_00007_nrcb2",
    "jw01180013001_10101_00006_nrcb2",
    "jw01180013001_10101_00003_nrcb2",
    "jw01180013001_10101_00001_nrcb2",
    "jw01180013001_10101_00008_nrcb2",
    "jw01180013001_10101_00004_nrcb2",
    "jw01180013001_10101_00005_nrcb2",
    "jw01180013001_06101_00005_nrcblong",
    "jw01180013001_06101_00004_nrcblong",
    "jw01180013001_06101_00006_nrcblong",
    "jw01180013001_06101_00002_nrcblong",
    "jw01180013001_06101_00008_nrcblong",
    "jw01180013001_06101_00009_nrcblong",
    "jw01180013001_06101_00001_nrcblong",
    "jw01180013001_06101_00007_nrcblong",
    "jw01180013001_06101_00003_nrcblong"
]



products_epoch1 = Observations.filter_products(prod,
                                        productSubGroupDescription=["CAL"],
                                        extension="fits",
                                        obs_id=epoch_1)

products_epoch2 = Observations.filter_products(prod,
                                        productSubGroupDescription=["CAL"],
                                        extension="fits",
                                        obs_id=epoch_2)

In [ ]:
manifest1 = Observations.download_products(products_epoch1, download_dir="./epoch1")
manifest2 = Observations.download_products(products_epoch2, download_dir="./epoch2")

In [2]:
# Get epoch 1 and epoch 2 cals in an array

epoch1_F356W_files = glob.glob("../JADES_data/epoch1/*/*/*/*nrcblong_cal.fits")
epoch1_F200W_files = glob.glob("../JADES_data/epoch1/*/*/*/*nrcb2_cal.fits")
epoch1_files = [] + epoch1_F356W_files + epoch1_F200W_files

epoch2_F356W_files = glob.glob("../JADES_data/epoch2/*/*/*/*nrcblong_cal.fits")
epoch2_F200W_files = glob.glob("../JADES_data/epoch2/*/*/*/*nrcb2_cal.fits")
epoch2_files = [] + epoch2_F356W_files + epoch2_F200W_files

# Get pre-made catalog file
ref_catname = "../JADES_data/jhat_cat.txt"
ref_fits = "../JADES_data/hlsp_jades_jwst_nircam_goods-s-deep_photometry_v2.0_catalog.fits"

# Make output dirs
jhat_epoch1_outsubdir = "epoch1_aligned"
jhat_epoch2_outsubdir = "epoch2_aligned"
os.makedirs(jhat_epoch1_outsubdir, exist_ok=True)
os.makedirs(jhat_epoch2_outsubdir, exist_ok=True)

In [2]:
def create_pixregionfile(x,y,regionname,color,coords='image',radius=1):
    if isinstance(radius,int):
        radius = [radius]*len(x)
    with open(regionname, 'w') as f:
        if isinstance(color,str):
            f.write('global color={0} dashlist=8 3 width=2 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n'.format(color))
            do_col = False
        else:
            do_col = True
            f.write('global dashlist=8 3 width=2 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
        f.write('%s \n'%coords)
        for star in range(len(x)):
            xval = x[star]
            yval = y[star]
            if do_col:
                f.write('circle({ra},{dec},{radius}") # color={color}\n'.format(ra=xval, dec=yval,radius=radius[star],color=color[star]))
            else:
                f.write('circle({ra},{dec},{radius}")\n'.format(ra=xval, dec=yval,radius=radius[star]))
    f.close()

In [4]:
wcs_align = st_wcs_align()
max_depth = 28.0

e1_cnt = len(epoch1_files)
for i, f in enumerate(epoch1_files):
    curr = i+1
    

    try:
        print("\t***** Processing File %s/%s: `%s` *****" % (curr, e1_cnt, f))
    
        wcs_align.run_all(f,
                      telescope='jwst',
                      outsubdir=jhat_epoch1_outsubdir,
                      refcat_racol='RA',
                      refcat_deccol='DEC',
                      refcat_magcol='MAG',
                      refcat_magerrcol='MAG_ERR',
                      overwrite=True,
                      d2d_max=0.25,
                      # showplots=2,
                      showplots=0,
                      refcatname=ref_catname,
                      histocut_order='dxdy',
                      sharpness_lim=(0.3,0.9), # 0 to 1
                      roundness1_lim=(-0.7, 0.7), # -1 to 1
                      SNR_min= 3,
                      dmag_max=1.0,
                      objmag_lim =(14,max_depth),
                      saveplots=2,
                      savephottable=True)
    
        good_matches_file_name = os.path.basename(f).replace('_cal.fits', '.goodmatches.csv')
        good_matches_file_path = "./{base_path}/{file_name}".format(base_path=jhat_epoch1_outsubdir, file_name=good_matches_file_name)
        good_match_tbl = Table.read(good_matches_file_path, format="ascii")

        create_pixregionfile(good_match_tbl["RA"], 
                             good_match_tbl["DEC"],
                             good_matches_file_path.replace("csv", "reg"), 
                             color="red", coords="icrs", radius=[0.4]*len(good_match_tbl))
        
              
    except Exception as e:
        print("\t\t***** FAILED Processing File %s/%s: `%s` *****" % (curr, e1_cnt, f))
        print(e)
        print("\t\tContinuing...")

        break # stop and figure out why


	***** Processing File 1/18: `../JADES_data/epoch1/mastDownload/JWST/jw01180015001_06101_00002_nrcblong/jw01180015001_06101_00002_nrcblong_cal.fits` *****
0 ./epoch1_aligned/jw01180015001_06101_00002_nrcblong.phot.txt


2025-08-07 10:33:29,307 - CRDS - INFO -  Calibration SW Found: jwst 1.19.1 (/opt/homebrew/Caskroom/miniconda/base/envs/jwst_ss/lib/python3.11/site-packages/jwst-1.19.1.dist-info)
model.meta.resample.pixel_scale_ratio was not found. Assuming the native detector pixel scale (i.e., pixel_scale_ratio = 1)


dmag 0.36200000000000004
DDD NIRCAM F356W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch1_aligned/jw01180015001_06101_00002_nrcblong.refcat.txt
reffile_d2d 0.25
    slope  intercept     maxval  index  d_bestguess  fwhm  multimax
-0.000098        0.1 249.709259   4051     0.120982  0.64     False
d_rot_tmp 0.9209821717160367
Keeping 341 out of 341, skippin 0 because of null values in columns d_rot_tmp
median: 0.119951
75.000000 percentile cut: max residual for cut: 0.205055
__tmp_residuals 0.20505480916924196
median: 0.124500
i:00 mean:0.124500(0.006053) stdev:0.096475(0.004272) X2norm:1.00 Nchanged:0 Ngood:255 Nclip:86

mean: 0.120094
i:01 mean:0.120094(0.007103) stdev:0.119068(0.005014) X2norm:1.00 Nchanged:27 Ngood:282 Nclip:59

mean: 0.121447
i:02 mean:0.121447(0.007935) stdev:0.136524(0.005602) X2norm:1.00 Nchanged:15 Ngood:297 Nclip:44

mean: 0.123711
i:03 mean:0.123711(0.009216) stdev:0.163825(0.00650

model.meta.resample.pixel_scale_ratio was not found. Assuming the native detector pixel scale (i.e., pixel_scale_ratio = 1)


dmag 0.36200000000000004
DDD NIRCAM F356W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch1_aligned/jw01180015001_06101_00003_nrcblong.refcat.txt
reffile_d2d 0.25
    slope  intercept     maxval  index  d_bestguess  fwhm  multimax
-0.000098        0.1 245.195864   4342     0.092658  0.64     False
d_rot_tmp 0.8926581049073434
Keeping 333 out of 333, skippin 0 because of null values in columns d_rot_tmp
median: 0.100919
75.000000 percentile cut: max residual for cut: 0.221274
__tmp_residuals 0.22127429467639756
median: 0.112830
i:00 mean:0.112830(0.006260) stdev:0.098587(0.004418) X2norm:1.00 Nchanged:0 Ngood:249 Nclip:84

mean: 0.104673
i:01 mean:0.104673(0.007696) stdev:0.129704(0.005433) X2norm:1.00 Nchanged:36 Ngood:285 Nclip:48

mean: 0.096490
i:02 mean:0.096490(0.009024) stdev:0.158622(0.006370) X2norm:1.00 Nchanged:25 Ngood:310 Nclip:23

mean: 0.093813
i:03 mean:0.093813(0.009293) stdev:0.164414(0.00656

...

dmag 0.36200000000000004
DDD NIRCAM F200W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch1_aligned/jw01180015001_10101_00006_nrcb2.refcat.txt
reffile_d2d 0.25
    slope  intercept    maxval  index  d_bestguess  fwhm  multimax
-0.000098        0.1 56.410606   5563     0.323784  0.88     False
d_rot_tmp 1.1237838042785484
Keeping 108 out of 108, skippin 0 because of null values in columns d_rot_tmp
median: 0.300002
75.000000 percentile cut: max residual for cut: 0.428564
__tmp_residuals 0.42856363155600924
median: 0.299235
i:00 mean:0.299235(0.022583) stdev:0.201984(0.015869) X2norm:0.99 Nchanged:0 Ngood:81 Nclip:27

mean: 0.300072
i:01 mean:0.300072(0.028306) stdev:0.275892(0.019911) X2norm:1.00 Nchanged:15 Ngood:96 Nclip:12

mean: 0.298356
i:02 mean:0.298356(0.034030) stdev:0.352007(0.023951) X2norm:1.00 Nchanged:12 Ngood:108 Nclip:0

mean: 0.298356
i:03 mean:0.298356(0.034030) stdev:0.352007(0.023951) X2nor

model.meta.resample.pixel_scale_ratio was not found. Assuming the native detector pixel scale (i.e., pixel_scale_ratio = 1)


dmag 0.36200000000000004
DDD NIRCAM F200W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch1_aligned/jw01180015001_10101_00008_nrcb2.refcat.txt
reffile_d2d 0.25
    slope  intercept    maxval  index  d_bestguess  fwhm  multimax
-0.000049       0.05 55.901548   3300     0.247828  0.98     False
d_rot_tmp 1.0478280827495923
Keeping 111 out of 111, skippin 0 because of null values in columns d_rot_tmp
median: 0.242870
75.000000 percentile cut: max residual for cut: 0.418916
__tmp_residuals 0.41891605429105233
median: 0.246385
i:00 mean:0.246385(0.023066) stdev:0.208875(0.016212) X2norm:0.99 Nchanged:0 Ngood:83 Nclip:28

mean: 0.229574
i:01 mean:0.229574(0.029131) stdev:0.292760(0.020497) X2norm:1.00 Nchanged:19 Ngood:102 Nclip:9

mean: 0.223575
i:02 mean:0.223575(0.033062) stdev:0.346759(0.023273) X2norm:1.00 Nchanged:9 Ngood:111 Nclip:0

mean: 0.223575
i:03 mean:0.223575(0.033062) stdev:0.346759(0.023273) X2norm

model.meta.resample.pixel_scale_ratio was not found. Assuming the native detector pixel scale (i.e., pixel_scale_ratio = 1)


dmag 0.36200000000000004
DDD NIRCAM F200W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch1_aligned/jw01180015001_10101_00009_nrcb2.refcat.txt
reffile_d2d 0.25
    slope  intercept   maxval  index  d_bestguess  fwhm  multimax
-0.000146       0.15 53.97238   5910     0.371887  1.12     False
d_rot_tmp 1.171886849500129
Keeping 113 out of 113, skippin 0 because of null values in columns d_rot_tmp
median: 0.326346
75.000000 percentile cut: max residual for cut: 0.460296
__tmp_residuals 0.46029608875824773
median: 0.360994
i:00 mean:0.360994(0.025722) stdev:0.234341(0.018080) X2norm:0.99 Nchanged:0 Ngood:84 Nclip:29

mean: 0.307628
i:01 mean:0.307628(0.031457) stdev:0.323864(0.022139) X2norm:1.00 Nchanged:23 Ngood:107 Nclip:6

mean: 0.298235
i:02 mean:0.298235(0.033767) stdev:0.357358(0.023771) X2norm:1.00 Nchanged:6 Ngood:113 Nclip:0

mean: 0.298235
i:03 mean:0.298235(0.033767) stdev:0.357358(0.023771) X2norm:1.

In [6]:
e2_cnt = len(epoch2_files)
for i, f in enumerate(epoch2_files):
    curr = i+1

    try:
        print("\t***** Processing File %s/%s: `%s` *****" % (curr, e1_cnt, f))
    
        wcs_align.run_all(f,
                      telescope='jwst',
                      outsubdir=jhat_epoch2_outsubdir,
                      refcat_racol='RA',
                      refcat_deccol='DEC',
                      refcat_magcol='MAG',
                      refcat_magerrcol='MAG_ERR',
                      overwrite=True,
                      d2d_max=0.25,
                      # showplots=2,
                      showplots=0,
                      refcatname=ref_catname,
                      histocut_order='dxdy',
                      sharpness_lim=(0.3,0.9), # 0 to 1
                      roundness1_lim=(-0.7, 0.7), # -1 to 1
                      SNR_min= 3,
                      dmag_max=1.0,
                      objmag_lim =(14,max_depth),
                      saveplots=2,
                      savephottable=True)
    
        good_matches_file_name = os.path.basename(f).replace('_cal.fits', '.goodmatches.csv')
        good_matches_file_path = "./{base_path}/{file_name}".format(base_path=jhat_epoch2_outsubdir, file_name=good_matches_file_name)
        good_match_tbl = Table.read(good_matches_file_path, format="ascii")

        create_pixregionfile(good_match_tbl["RA"], 
                             good_match_tbl["DEC"],
                             good_matches_file_path.replace("csv", "reg"), 
                             color="red", coords="icrs", radius=[0.4]*len(good_match_tbl))
        
              
    except Exception as e:
        print("\t\t***** FAILED Processing File %s/%s: `%s` *****" % (curr, e1_cnt, f))
        print(e)
        print("\t\tContinuing...")

        break # stop and figure out why

	***** Processing File 1/18: `../JADES_data/epoch2/mastDownload/JWST/jw01180013001_06101_00007_nrcblong/jw01180013001_06101_00007_nrcblong_cal.fits` *****
0 ./epoch2_aligned/jw01180013001_06101_00007_nrcblong.phot.txt


model.meta.resample.pixel_scale_ratio was not found. Assuming the native detector pixel scale (i.e., pixel_scale_ratio = 1)


dmag 0.36200000000000004
DDD NIRCAM F356W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch2_aligned/jw01180013001_06101_00007_nrcblong.refcat.txt
reffile_d2d 0.25
    slope  intercept     maxval  index  d_bestguess  fwhm  multimax
-0.000098        0.1 209.355248   4659    -0.316478  0.66     False
d_rot_tmp 0.4835224069040749
Keeping 298 out of 298, skippin 0 because of null values in columns d_rot_tmp
median: -0.316602
75.000000 percentile cut: max residual for cut: 0.243935
__tmp_residuals 0.24393509481287534
median: -0.318222
i:00 mean:-0.318222(0.007507) stdev:0.111856(0.005297) X2norm:1.00 Nchanged:0 Ngood:223 Nclip:75

mean: -0.312958
i:01 mean:-0.312958(0.009078) stdev:0.144967(0.006407) X2norm:1.00 Nchanged:33 Ngood:256 Nclip:42

mean: -0.306101
i:02 mean:-0.306101(0.009907) stdev:0.161586(0.006993) X2norm:1.00 Nchanged:11 Ngood:267 Nclip:31

mean: -0.301229
i:03 mean:-0.301229(0.010651) stdev:0.17597

model.meta.resample.pixel_scale_ratio was not found. Assuming the native detector pixel scale (i.e., pixel_scale_ratio = 1)


dmag 0.36200000000000004
DDD NIRCAM F356W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch2_aligned/jw01180013001_06101_00006_nrcblong.refcat.txt
reffile_d2d 0.25
    slope  intercept     maxval  index  d_bestguess  fwhm  multimax
-0.000098        0.1 226.772156   3788    -0.309522  0.62     False
d_rot_tmp 0.4904781808828773
Keeping 310 out of 310, skippin 0 because of null values in columns d_rot_tmp
median: -0.316503
75.000000 percentile cut: max residual for cut: 0.209933
__tmp_residuals 0.2099327595650584
median: -0.317587
i:00 mean:-0.317587(0.006195) stdev:0.094154(0.004371) X2norm:1.00 Nchanged:0 Ngood:232 Nclip:78

mean: -0.318237
i:01 mean:-0.318237(0.007347) stdev:0.117092(0.005185) X2norm:1.00 Nchanged:23 Ngood:255 Nclip:55

mean: -0.317888
i:02 mean:-0.317888(0.008226) stdev:0.134669(0.005806) X2norm:1.00 Nchanged:14 Ngood:269 Nclip:41

mean: -0.314934
i:03 mean:-0.314934(0.009147) stdev:0.153056

...

dmag 0.36200000000000004
DDD NIRCAM F200W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch2_aligned/jw01180013001_10101_00007_nrcb2.refcat.txt
reffile_d2d 0.25
    slope  intercept    maxval  index  d_bestguess  fwhm  multimax
-0.000146       0.15 59.840295   7622    -0.479942  0.92     False
d_rot_tmp 0.3200577792340369
Keeping 118 out of 118, skippin 0 because of null values in columns d_rot_tmp
median: -0.486053
75.000000 percentile cut: max residual for cut: 0.401739
__tmp_residuals 0.40173907861036334
median: -0.486053
i:00 mean:-0.486053(0.022350) stdev:0.208463(0.015714) X2norm:0.99 Nchanged:0 Ngood:88 Nclip:30

mean: -0.484857
i:01 mean:-0.484857(0.027729) stdev:0.282786(0.019514) X2norm:1.00 Nchanged:17 Ngood:105 Nclip:13

mean: -0.492205
i:02 mean:-0.492205(0.033174) stdev:0.358834(0.023358) X2norm:1.00 Nchanged:13 Ngood:118 Nclip:0

mean: -0.492205
i:03 mean:-0.492205(0.033174) stdev:0.358834(0.023

model.meta.resample.pixel_scale_ratio was not found. Assuming the native detector pixel scale (i.e., pixel_scale_ratio = 1)


dmag 0.36200000000000004
DDD NIRCAM F200W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch2_aligned/jw01180013001_10101_00003_nrcb2.refcat.txt
reffile_d2d 0.25
    slope  intercept   maxval  index  d_bestguess  fwhm  multimax
-0.000293        0.3 50.13635  10858    -0.507867  1.04     False
d_rot_tmp 0.2921325940424879
Keeping 101 out of 101, skippin 0 because of null values in columns d_rot_tmp
median: -0.515045
75.000000 percentile cut: max residual for cut: 0.423557
__tmp_residuals 0.4235569929905384
median: -0.491403
i:00 mean:-0.491403(0.025418) stdev:0.218654(0.017853) X2norm:0.99 Nchanged:0 Ngood:75 Nclip:26

mean: -0.520111
i:01 mean:-0.520111(0.030898) stdev:0.297969(0.021732) X2norm:1.00 Nchanged:19 Ngood:94 Nclip:7

mean: -0.554225
i:02 mean:-0.554225(0.034042) stdev:0.340422(0.023952) X2norm:1.00 Nchanged:7 Ngood:101 Nclip:0

mean: -0.554225
i:03 mean:-0.554225(0.034042) stdev:0.340422(0.023952) X

model.meta.resample.pixel_scale_ratio was not found. Assuming the native detector pixel scale (i.e., pixel_scale_ratio = 1)


dmag 0.36200000000000004
DDD NIRCAM F200W CLEAR ggg None
####### coron None
dmag 1.0
sharpness 0.9
roundness1 0.7
mag 28.0
x 2008
y 2008
Saving refcat file into ./epoch2_aligned/jw01180013001_10101_00001_nrcb2.refcat.txt
reffile_d2d 0.25
    slope  intercept    maxval  index  d_bestguess  fwhm  multimax
-0.000195        0.2 56.755579   7643    -0.527691  0.94     False
d_rot_tmp 0.2723092947022565
Keeping 112 out of 112, skippin 0 because of null values in columns d_rot_tmp
median: -0.532731
75.000000 percentile cut: max residual for cut: 0.374945
__tmp_residuals 0.3749448569762587
median: -0.532731
i:00 mean:-0.532731(0.022571) stdev:0.205631(0.015865) X2norm:0.99 Nchanged:0 Ngood:84 Nclip:28

mean: -0.516634
i:01 mean:-0.516634(0.027860) stdev:0.278599(0.019602) X2norm:1.00 Nchanged:17 Ngood:101 Nclip:11

mean: -0.527760
i:02 mean:-0.527760(0.032742) stdev:0.344955(0.023048) X2norm:1.00 Nchanged:11 Ngood:112 Nclip:0

mean: -0.527760
i:03 mean:-0.527760(0.032742) stdev:0.344955(0.0230

In [7]:
aligned_epoch1_F356W_files = glob.glob("%s/*nrcblong_jhat.fits" % (jhat_epoch1_outsubdir))
aligned_epoch1_F200W_files = glob.glob("%s/*nrcb2_jhat.fits" % (jhat_epoch1_outsubdir))

aligned_epoch2_F356W_files = glob.glob("%s/*nrcblong_jhat.fits" % (jhat_epoch2_outsubdir))
aligned_epoch2_F200W_files = glob.glob("%s/*nrcb2_jhat.fits" % (jhat_epoch2_outsubdir))

# Choose an appropriate projection
pprint.pprint(aligned_epoch1_F356W_files)
pprint.pprint(aligned_epoch2_F200W_files)

sw_pix_scale = 30.0 / 1000 # milliarcseconds
lw_pix_scale = 60.0 / 1000 # milliarcseconds

# Positioned between both SNe
CRVAL1 = 53.1359270
CRVAL2 = -27.8174387

delta_ra = 0.01 # degrees
delta_dec = 0.01 # degrees

# tile_shape = np.array((delta_ra, delta_dec)) * 3600 / lw_pix_scale
tile_shape = np.array((2048, 2048))

CRPIX1, CRPIX2 = tile_shape / 2
rotation = 0

print(tile_shape)

['epoch1_aligned/jw01180015001_06101_00007_nrcblong_jhat.fits',
 'epoch1_aligned/jw01180015001_06101_00004_nrcblong_jhat.fits',
 'epoch1_aligned/jw01180015001_06101_00009_nrcblong_jhat.fits',
 'epoch1_aligned/jw01180015001_06101_00002_nrcblong_jhat.fits',
 'epoch1_aligned/jw01180015001_06101_00001_nrcblong_jhat.fits',
 'epoch1_aligned/jw01180015001_06101_00003_nrcblong_jhat.fits',
 'epoch1_aligned/jw01180015001_06101_00005_nrcblong_jhat.fits',
 'epoch1_aligned/jw01180015001_06101_00008_nrcblong_jhat.fits',
 'epoch1_aligned/jw01180015001_06101_00006_nrcblong_jhat.fits']
['epoch2_aligned/jw01180013001_10101_00008_nrcb2_jhat.fits',
 'epoch2_aligned/jw01180013001_10101_00007_nrcb2_jhat.fits',
 'epoch2_aligned/jw01180013001_10101_00001_nrcb2_jhat.fits',
 'epoch2_aligned/jw01180013001_10101_00009_nrcb2_jhat.fits',
 'epoch2_aligned/jw01180013001_10101_00006_nrcb2_jhat.fits',
 'epoch2_aligned/jw01180013001_10101_00003_nrcb2_jhat.fits',
 'epoch2_aligned/jw01180013001_10101_00004_nrcb2_jhat.fits

In [8]:
pipe3 = Image3Pipeline()

ep1_output_dir = "./epoch1_mosaic/"
ep2_output_dir = "./epoch2_mosaic/"
os.makedirs(ep1_output_dir, exist_ok=True)
os.makedirs(ep2_output_dir, exist_ok=True)

    
# For SW first:
params = {
            'assign_mtwcs':         {'skip': True},
            'tweakreg':             {'skip': True},
            'outlier_detection':    {'skip': True},
            'source_catalog':       {'skip': True},
            'skymatch':             {'skip': True}
            # 'skymatch':             {'skymethod':'global+match'}
        }

params['resample'] = {  
                        'skip': False,
                        'pixfrac': 1.,
                        'kernel': 'square',
                        'fillval': 'indef',
                        'weight_type': 'ivm', # 'exptime',
                        'in_memory': True,
                        'save_results': True
                    }

params['resample']['pixel_scale'] = sw_pix_scale  
params['resample']['rotation'] = rotation
params['resample']['output_shape'] = (int(tile_shape[0]), int(tile_shape[1]))

params['resample']['crpix'] = [CRPIX1, CRPIX2]
params['resample']['crval'] = [CRVAL1, CRVAL2]

pipe3.call([img for img in aligned_epoch1_F200W_files], 
           steps=params, 
           output_dir=ep1_output_dir, 
           output_file="f200w_epoch1", 
           save_results=True)

pipe3.call([img for img in aligned_epoch2_F200W_files], 
           steps=params, 
           output_dir=ep2_output_dir, 
           output_file="f200w_epoch2", 
           save_results=True)


2025-08-07 10:38:23,633 - stpipe - INFO - PARS-TWEAKREGSTEP parameters found: /grp/crds/cache/references/jwst/jwst_nircam_pars-tweakregstep_0036.asdf
2025-08-07 10:38:23,651 - stpipe - INFO - PARS-RESAMPLESTEP parameters found: /grp/crds/cache/references/jwst/jwst_nircam_pars-resamplestep_0001.asdf
2025-08-07 10:38:23,665 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /grp/crds/cache/references/jwst/jwst_nircam_pars-sourcecatalogstep_0007.asdf
2025-08-07 10:38:23,677 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2025-08-07 10:38:23,678 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2025-08-07 10:38:23,679 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2025-08-07 10:38:23,680 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2025-08-07 10:38:23,680 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2025-08-07 10:38:23,681 - stpipe.Image

In [48]:
# For LW next:
params['resample']['pixel_scale'] = lw_pix_scale  


pipe3.call([img for img in aligned_epoch1_F356W_files], 
           steps=params, 
           output_dir=ep1_output_dir, 
           output_file="f356w_epoch1", 
           save_results=True)

pipe3.call([img for img in aligned_epoch2_F356W_files], 
           steps=params, 
           output_dir=ep2_output_dir, 
           output_file="f356w_epoch2", 
           save_results=True)


2025-08-07 15:15:01,901 - stpipe - INFO - PARS-TWEAKREGSTEP parameters found: /grp/crds/cache/references/jwst/jwst_nircam_pars-tweakregstep_0043.asdf
2025-08-07 15:15:01,935 - stpipe - INFO - PARS-RESAMPLESTEP parameters found: /grp/crds/cache/references/jwst/jwst_nircam_pars-resamplestep_0001.asdf
2025-08-07 15:15:01,951 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /grp/crds/cache/references/jwst/jwst_nircam_pars-sourcecatalogstep_0029.asdf
2025-08-07 15:15:01,971 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2025-08-07 15:15:01,972 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2025-08-07 15:15:01,975 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2025-08-07 15:15:01,976 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2025-08-07 15:15:01,977 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2025-08-07 15:15:01,978 - stpipe.Image

In [35]:
import re

if match:
    epoch = 
    print(epoch)

epoch1


In [50]:
def run_hotpants(inst_name, filt_name, ref_input_file, sci_input_file, cat_file, diff_dir):

    filters = {}
    filters['NIRCAM'] = {'F070W': 0.987, 'F090W': 1.103, 'F115W': 1.298, 'F140M': 1.553, 'F150W2': 1.628,
                         'F150W': 1.770,
                         'F162M': 1.801, 'F164N': 1.494, 'F182M': 1.990, 'F187N': 2.060, 'F200W': 2.141, 'F210M': 2.304,
                         'F212N': 2.341, 'F250M': 1.340, 'F277W': 1.444, 'F300M': 1.585, 'F322W2': 1.547,
                         'F323N': 1.711,
                         'F335M': 1.760, 'F356W': 1.830, 'F360M': 1.901, 'F405N': 2.165, 'F410M': 2.179, 'F430M': 2.300,
                         'F444W': 2.302, 'F460M': 2.459, 'F466N': 2.507, 'F470N': 2.535, 'F480M': 2.574}
    filters['NIRISS'] = {'F090W': 1.40, 'F115W': 1.40, 'F140M': 1.50, 'F150W': 1.50, 'F158M': 1.50, 'F200W': 1.50,
                         'F277W': 1.50, 'F356W': 1.60, 'F380M': 1.70, 'F430M': 1.80, 'F444W': 1.80, 'F480M': 1.80}
    filters['MIRI'] = {'F560W': 1.636, 'F770W': 2.187, 'F1000W': 2.888, 'F1130W': 3.318, 'F1280W': 3.713,
                       'F1500W': 4.354, 'F1800W': 5.224, 'F2100W': 5.989, 'F2550W': 7.312}
    filters['WFC3'] = {'F105W': 1.001, 'F110W': 1.019, 'F125W': 1.053, 'F140W': 1.100, 'F160W': 1.176}
    filters['ACS'] = {'F814W': 1.0}

    def unpack_fits(input_file, sci_out_filename, err_out_filename):
        
        # Unpack data into single extension files
        with fits.open(input_file, output_verify='fix') as dat:
    
            sci_data = dat['SCI', 1].data
            max_sci_val = np.nanmax(sci_data)
        
            err_data = dat['ERR', 1].data
            err_median = np.nanmedian(err_data)
            sci_pedastal = 50.0 * err_median
            print("\n\tMedian error value: `%0.4f`; Pedastal to add to Sci Im: `%0.4f`" % (err_median, sci_pedastal))
        
            dat[0].data = sci_data + sci_pedastal
            dat[0].header = dat[0].header + dat[1].header
            dat[0].writeto(sci_out_filename, overwrite=True)
        
            dat[0].data = err_data
            dat[0].writeto(err_out_filename, overwrite=True)

            return max_sci_val

    def create_mask(input_err_file, output_mask_file):

        with fits.open(input_err_file, output_verify='fix') as dat:
            sci_mask = np.zeros_like(dat[0].data)
            sci_mask[dat[0].data == 0] = 0x80
            sci_mask[np.isnan(dat[0].data)] = 0x80
        
            dat[0].data = sci_mask
            dat[0].scale('int16')
            dat.writeto(output_mask_file, overwrite=True)


    # Takes Tabular Fits file w/ specific columns... feel free to generalize
    def create_stamp_catalog(reference_cat_fits, input_file, filt_name, flux_thresh, stamp_outfile):
        
        # Build stampxy file
        with fits.open(reference_cat_fits, memmap=True) as cat_file:
        
            table = Table(cat_file["CIRC_BSUB"].data)            
            
            flux_key = "{filt}_CIRC0".format(filt=filt_name)
            err_key = "{filt}_CIRC0_e".format(filt=filt_name)
            
            ras = table["RA"]
            decs = table["DEC"]
        
            flux = table[flux_key]
            flux_err = table[err_key]

            # reverse sort so brightest sources first
            indices = (-flux).argsort()
        
            sorted_ra = ras[indices]
            sorted_dec = decs[indices]
            sorted_fluxes = flux[indices]
            sorted_flux_err = flux_err[indices]

            # get sources brighter than some threshold
            bright_ind = np.where(sorted_fluxes > flux_thresh)[0]
            bright_ra = sorted_ra[bright_ind]
            bright_dec = sorted_dec[bright_ind]
            bright_flux = sorted_fluxes[bright_ind]
            bright_flux_err = sorted_flux_err[bright_ind]
                 
            coords = SkyCoord(bright_ra, bright_dec, unit=(u.deg, u.deg))
            sci_obs3 = space_phot.observation3(input_file)
            y_max, x_max = sci_obs3.data.shape
            xs, ys = sci_obs3.wcs.world_to_pixel(coords)
            in_img_indices = np.where((xs >= 0) & (xs <= x_max) & (ys >= 0) & (ys <= y_max))[0]
            
            output_tbl = Table(
                [xs[in_img_indices],
                 ys[in_img_indices],
                 bright_ra[in_img_indices],
                 bright_dec[in_img_indices],
                 bright_flux[in_img_indices]], names=["X", "Y", "RA", "DEC", filt_name])
        
            output_tbl.write(stamp_outfile, format="ascii", overwrite=True)
            create_pixregionfile(xs[in_img_indices], ys[in_img_indices], stamp_outfile.replace("txt", "reg"), color="red",
                                 coords="image", radius=[0.4] * len(ys[in_img_indices]))

    
    # Create single extension file names
    sci_output_file = sci_input_file.replace(".fits", "_1.fits")
    sci_output_err_file = sci_input_file.replace(".fits", "_1.noise.fits")
    sci_output_mask_file = sci_input_file.replace(".fits", "_1.mask.fits")
    sci_stampxy_file = sci_input_file.replace('.fits', '.stampxy.txt')
    max_val_sci_im = unpack_fits(sci_input_file, sci_output_file, sci_output_err_file)

    ref_output_file = ref_input_file.replace(".fits", "_1.fits")
    ref_output_err_file = ref_input_file.replace(".fits", "_1.noise.fits")
    ref_output_mask_file = ref_input_file.replace(".fits", "_1.mask.fits")
    max_val_ref_im = unpack_fits(ref_input_file, ref_output_file, ref_output_err_file)
        
    create_mask(sci_output_err_file, sci_output_mask_file)
    create_mask(ref_output_err_file, ref_output_mask_file)

    diff_file_basename_formatter = "{filt_name}_{sci_epoch}_{ref_epoch}.diff.fits"
    epoch_regex = r'epoch\d+'

    match = re.search(epoch_regex, ref_input_file)
    ref_file_epoch = match.group(0)
    
    match = re.search(epoch_regex, sci_input_file)
    sci_file_epoch = match.group(0)
    
    diff_file = os.path.join(diff_dir, 
                             diff_file_basename_formatter.format(filt_name=filt_name, sci_epoch=sci_file_epoch, ref_epoch=ref_file_epoch))
    diff_err_file = diff_file.replace('.fits', '.noise.fits')
    diff_mask_file = diff_file.replace('.fits', '.mask.fits')
    diff_kernel_file = diff_file.replace('.fits', '.kernel.fits') 
    diff_stampxy_reg_file = diff_file.replace('.fits', '.stampxy.reg')

    create_stamp_catalog(cat_file, sci_output_file, filt_name, flux_thresh=10.0, stamp_outfile=sci_stampxy_file)

    # A bit dodgey: We're assuming the same instrument and filter between science and template image
    sci_fwhm = 1.8
    if inst_name in filters:
        if filt_name in filters[inst_name]:
            sci_fwhm = filters[inst_name][filt_name]
    else:
        print(f'{inst_name} not found in {list(filters.keys())}, FWHM fixed to 1.8 pixels.')

    temp_fwhm = 1.8
    if inst_name in filters:
        if filt_name in filters[inst_name]:
            temp_fwhm = filters[inst_name][filt_name]
    else:
        print(f'{inst_name} not found in {list(filters.keys())}, FWHM fixed to 1.8 pixels.')


    # hotpants args
    ko = 2  # spatial order of kernel variation within region (2)
    bgo = 2  # spatial order of background variation within region (1)
    ssig = 3.0  # threshold for sigma clipping statistics  (3.0)
    
    ks = 2.0  # high sigma rejection for bad stamps in kernel fit (2.0)
    r = 2.5 * sci_fwhm  # FWHM * 2.5         # convolution kernel half width (10)
    kfm = 0.99  # fraction of abs(kernel) sum for ok pixel (0.990)
    
    il = 0  # lower valid data count, image (0)
    iu = max_val_sci_im  # dynamically take the highest in the science img        # upper valid data count, image (25000)
    iuk = iu  # set to 'iu' for now     # upper valid data count for kernel, image (iuthresh)
    
    tl = 0  # lower valid data count, template (0)
    tu = max_val_ref_im  # upper valid data count, template (25000)
    tuk = tu  # upper valid data count for kernel, template (tuthresh)
    
    nrx = 1 #1  # number of image regions in x dimension (1)
    nry = 1 #1  # number of image regions in y dimension (1)
    
    nsx = 8 #70  # number of each region's stamps in x dimension (10)
    nsy = 8 #90  # number of each region's stamps in y dimension (10)
    nss = 7  # number of centroids to use for each stamp (3) # DC: got a segmentation fault using "10"
    
    # ngauss degree0 sigma0 .. degreeN sigmaN]
    # : ngauss = number of gaussians which compose kernel (3)
    # : degree = degree of polynomial associated with gaussian #
    #            (6 4 2)
    # : sigma  = width of gaussian #
    #            (0.70 1.50 3.00)
    # : N = 0 .. ngauss - 1
    
    # : (3 6 0.70 4 1.50 2 3.00
    ng = (3, 6, sci_fwhm / 2.0, 4, sci_fwhm, 2, sci_fwhm * 2.0)
    
    rss = 2.5 * sci_fwhm  # 2.5 * FWHM scaled by the fwhm - radius of the substamp # half width substamp to extract around each centroid (15)
    ft = 5.0  # RMS threshold for good centroid in kernel fit (20.0)
    
    # -okn          # rescale noise for 'ok' pixels (0)
    c = "t"  # force convolution on (t)emplate or (i)mage (undef)
    n = "i"  # normalize to (t)emplate, (i)mage, or (u)nconvolved (t)
    # -sconv        # all regions convolved in same direction (0)
    
    afssc = 0  # autofind stamp centers so #=-nss when -ssf,-cmp (1)
    gridssc = 0
    
    fi = 0.0  # value for invalid (bad) pixels (1.0e-30)
    fin = 0.0  # noise image only fillvalue (0.0e+00)
    
    mins = 2.0  # Fraction of kernel half width to spread input mask (1.0)
    mous = 0.0  # Ditto output mask, negative = no diffim masking (1.0)
    
    v = 2  # level of verbosity, 0-2 (1)
    # -savexy = set file name . saves the X,Y positions of used, clipped, and all substamps in different colors
    # save as a reg file
    
    hotpants_arg = 'hotpants -inim {sci_file} -tmplim {temp_file} -outim {diff_file} -ini {sci_noise_im} \
                   -imi {sci_mask} -il {il} -iu {iu} -iuk {iuk} -tni {temp_noise_im} -tmi {temp_mask} \
                   -tl {tl} -tu {tu} -tuk {tuk} -nrx {nrx} -nry {nry} -nsx {nsx} -nsy {nsy} -nss {nss} -ng {ng_tuple} \
                   -rss {rss} -ft {ft} -r {r} -ko {ko} -bgo {bgo} -ssig {ssig} -ks {ks} -kfm {kfm} -okn -c {c} -n {n} -sconv \
                   -cmp {diff_substamps} -afssc {afssc} -gridssc {gridssc} -fi {fi} \
                   -oni {diff_noise_im}  -fin {fin} -mins {mins} \
                   -omi {diff_mask} -mous {mous} -oki {diff_kernel} -v {v} -savexy {savexy}'.format(
        sci_file=sci_output_file, temp_file=ref_output_file, diff_file=diff_file,
        sci_noise_im=sci_output_err_file, sci_mask=sci_output_mask_file, il=il, iu=iu, iuk=iuk, temp_noise_im=ref_output_err_file,
        temp_mask=ref_output_mask_file, tl=tl, tu=tu, tuk=tuk, nrx=nrx, nry=nry, nsx=nsx, nsy=nsy, nss=nss,
        ng_tuple=" ".join([str(_ng) for _ng in ng]),
        rss=rss, ft=ft, r=r, ko=ko, bgo=bgo, ssig=ssig, ks=ks, kfm=kfm, c=c, n=n, diff_substamps=sci_stampxy_file,
        afssc=afssc, gridssc=gridssc, fi=fi, diff_noise_im=diff_err_file, fin=fin,
        mins=mins, diff_mask=diff_mask_file, mous=mous, diff_kernel=diff_kernel_file, v=v, savexy=diff_stampxy_reg_file)
    
    print("Hotpants invocation:\n\t%s" % hotpants_arg)

    os.system("docker run --rm -v /Users/dcoulter/Documents/JWST_Summer_School/Difference_Imaging:/app ghcr.io/davecoulter/diffpype:1.0.3 %s" % hotpants_arg)
    # process = subprocess.Popen(hotpants_arg, shell=True)    


---

In [51]:
# def run_hotpants(inst_name, filt_name, ref_input_file, sci_input_file, cat_file, diff_dir):
run_hotpants(inst_name="NIRCAM", 
             filt_name="F200W", 
             ref_input_file="./epoch1_mosaic/f200w_epoch1_i2d.fits", 
             sci_input_file="./epoch2_mosaic/f200w_epoch2_i2d.fits", 
             cat_file=ref_fits, 
             diff_dir="./diff")


	Median error value: `0.0040`; Pedastal to add to Sci Im: `0.2007`

	Median error value: `0.0041`; Pedastal to add to Sci Im: `0.2044`
Hotpants invocation:
	hotpants -inim ./epoch2_mosaic/f200w_epoch2_i2d_1.fits -tmplim ./epoch1_mosaic/f200w_epoch1_i2d_1.fits -outim ./diff/F200W_epoch2_epoch1.diff.fits -ini ./epoch2_mosaic/f200w_epoch2_i2d_1.noise.fits                    -imi ./epoch2_mosaic/f200w_epoch2_i2d_1.mask.fits -il 0 -iu 14.31140422821045 -iuk 14.31140422821045 -tni ./epoch1_mosaic/f200w_epoch1_i2d_1.noise.fits -tmi ./epoch1_mosaic/f200w_epoch1_i2d_1.mask.fits                    -tl 0 -tu 14.069025993347168 -tuk 14.069025993347168 -nrx 1 -nry 1 -nsx 8 -nsy 8 -nss 7 -ng 3 6 1.0705 4 2.141 2 4.282                    -rss 5.3525 -ft 5.0 -r 5.3525 -ko 2 -bgo 2 -ssig 3.0 -ks 2.0 -kfm 0.99 -okn -c t -n i -sconv                    -cmp ./epoch2_mosaic/f200w_epoch2_i2d.stampxy.txt -afssc 0 -gridssc 0 -fi 0.0                    -oni ./diff/F200W_epoch2_epoch1.diff.noise.fits  -fin 0.0

Doing : ./epoch2_mosaic/f200w_epoch2_i2d_1.fits -
        ./epoch1_mosaic/f200w_epoch1_i2d_1.fits =
        ./diff/F200W_epoch2_epoch1.diff.fits
   Good templ data : 0.0 -> 14.1
   Good image data : 0.0 -> 14.3
Mallocing massive amounts of memory...
Region 0 pixels            : 1:2048,1:2048
 Vector Indices (buffered) : 0:2047,0:2047
 Vector Indices (good data): 0:2047,0:2047
Allocating image stamps...
Allocating template stamps...
region size: (X,Y)=(2048,2048)
# stamps: (X,Y)=(8,8)
stamp size: (X,Y)=(255,255)
Build stamp  : t    0 i    0 (grid coord  0  0)

Adding centers manually to s(0,0) x:0-254 y:0-254
X stamp: 0-254 region:0-2047
Y stamp: 0-254 region:0-2047
    Stamp in region : 0:254,0:254 X=77.0 Y=123.6
    Stamp in region : 0:254,0:254 X=59.1 Y=232.5
    Stamp in region : 0:254,0:254 X=117.1 Y=221.9
    Stamp in region : 0:254,0:254 X=41.7 Y=202.0
    templ: 0 substamps
Build stamp  : t    0 i    0 (grid coord  1  0)

Adding centers manually to s(1,0) x:256-510 y:0-254
X sta

In [52]:
run_hotpants(inst_name="NIRCAM", 
             filt_name="F356W", 
             ref_input_file="./epoch1_mosaic/f356w_epoch1_i2d.fits", 
             sci_input_file="./epoch2_mosaic/f356w_epoch2_i2d.fits", 
             cat_file=ref_fits, 
             diff_dir="./diff")


	Median error value: `0.0014`; Pedastal to add to Sci Im: `0.0700`

	Median error value: `0.0014`; Pedastal to add to Sci Im: `0.0706`
Hotpants invocation:
	hotpants -inim ./epoch2_mosaic/f356w_epoch2_i2d_1.fits -tmplim ./epoch1_mosaic/f356w_epoch1_i2d_1.fits -outim ./diff/F356W_epoch2_epoch1.diff.fits -ini ./epoch2_mosaic/f356w_epoch2_i2d_1.noise.fits                    -imi ./epoch2_mosaic/f356w_epoch2_i2d_1.mask.fits -il 0 -iu 26.41910171508789 -iuk 26.41910171508789 -tni ./epoch1_mosaic/f356w_epoch1_i2d_1.noise.fits -tmi ./epoch1_mosaic/f356w_epoch1_i2d_1.mask.fits                    -tl 0 -tu 27.60917091369629 -tuk 27.60917091369629 -nrx 1 -nry 1 -nsx 8 -nsy 8 -nss 7 -ng 3 6 0.915 4 1.83 2 3.66                    -rss 4.575 -ft 5.0 -r 4.575 -ko 2 -bgo 2 -ssig 3.0 -ks 2.0 -kfm 0.99 -okn -c t -n i -sconv                    -cmp ./epoch2_mosaic/f356w_epoch2_i2d.stampxy.txt -afssc 0 -gridssc 0 -fi 0.0                    -oni ./diff/F356W_epoch2_epoch1.diff.noise.fits  -fin 0.0 -mins 

Doing : ./epoch2_mosaic/f356w_epoch2_i2d_1.fits -
        ./epoch1_mosaic/f356w_epoch1_i2d_1.fits =
        ./diff/F356W_epoch2_epoch1.diff.fits
   Good templ data : 0.0 -> 27.6
   Good image data : 0.0 -> 26.4
Mallocing massive amounts of memory...
Region 0 pixels            : 1:2048,1:2048
 Vector Indices (buffered) : 0:2047,0:2047
 Vector Indices (good data): 0:2047,0:2047
Allocating image stamps...
Allocating template stamps...
region size: (X,Y)=(2048,2048)
# stamps: (X,Y)=(8,8)
stamp size: (X,Y)=(255,255)
Build stamp  : t    0 i    0 (grid coord  0  0)

Adding centers manually to s(0,0) x:0-254 y:0-254
X stamp: 0-254 region:0-2047
Y stamp: 0-254 region:0-2047
    Stamp in region : 0:254,0:254 X=143.4 Y=6.5
    Stamp in region : 0:254,0:254 X=207.9 Y=184.7
    Stamp in region : 0:254,0:254 X=85.4 Y=95.1
    Stamp in region : 0:254,0:254 X=51.3 Y=101.5
    Stamp in region : 0:254,0:254 X=109.1 Y=103.5
    Stamp in region : 0:254,0:254 X=127.3 Y=198.7
    Stamp in region : 0:254,0:2

In [19]:
# Run Straight Diffs
straight_sci = fits.open("./epoch2_mosaic/f200w_epoch2_i2d.fits")
straight_ref = fits.open("./epoch1_mosaic/f200w_epoch1_i2d.fits")

straight_sci['SCI'].data -= straight_ref['SCI'].data
straight_sci.writeto("./diff/F200W_epoch2_epoch1.straight_diff.fits",overwrite=True)

straight_sci['SCI'].data = np.sqrt(straight_ref['ERR'].data**2 + straight_sci['ERR'].data**2)
straight_sci.writeto("./diff/F200W_epoch2_epoch1.straight_diff.noise.fits",overwrite=True)



# Run Photutils

# Run SExtractor




In [22]:
# detection_limit=28, # in AB mag
snr_thresh = 3.0
# class_star_cut = 0

sew = sewpy.SEW(params=["X_IMAGE", "Y_IMAGE", "FLUX_RADIUS(3)", "FLAGS", "XPEAK_WORLD", "YPEAK_WORLD","CLASS_STAR"],
                    config={"DETECT_MINAREA":3, "PHOT_FLUXFRAC":"0.3, 0.5, 0.8",'DETECT_THRESH':snr_thresh,
                            'BACKPHOTO_TYPE':'local'},loglevel=0)

diffim = './diff/F200W_epoch2_epoch1.diff.fits'
diff_wcs = WCS(fits.open(diffim)[0])

found = sew(diffim)['table']
found['xcentroid'] = found['X_IMAGE'] -1 # zero based to one based array fix
found['ycentroid'] = found['Y_IMAGE'] -1
found['skycoord_peak'] = diff_wcs.pixel_to_world(found['xcentroid'],found['ycentroid'])
# found = found[found['CLASS_STAR']>class_star_cut]

table_cols = ['source_id','ra','dec']
table_data = [
    ['src_'+str(x) for x in np.arange(0,len(found),1)],
    [x.ra.value for x in found['skycoord_peak']],
    [x.dec.value for x in found['skycoord_peak']]
]

source_cat = Table(names=table_cols,dtype=[str]+[float]+[float])

for data_row in zip(*table_data):
    new_row = {}
    for i, d in enumerate(data_row):
        new_row[table_cols[i]] = d
    source_cat.add_row(new_row)

source_cat.write("./diff/F200W_epoch2_epoch1.sources.txt",format='ascii',overwrite=True)
create_pixregionfile(source_cat['ra'], source_cat['dec'], './diff/F200W_epoch2_epoch1.sources.reg', color="red",
                                 coords="icrs", radius=[0.4] * len(source_cat))

Ouch, SExtractor complains :
b''


In [14]:
detection_limit=28, # in AB mag
snr_thresh = 3.0
sew = sewpy.SEW(params=["X_IMAGE", "Y_IMAGE", "FLUX_RADIUS(3)", "FLAGS", "XPEAK_WORLD", "YPEAK_WORLD","CLASS_STAR"],
                    config={"DETECT_MINAREA":3, "PHOT_FLUXFRAC":"0.3, 0.5, 0.8",'DETECT_THRESH':snr_thresh,
                            'BACKPHOTO_TYPE':'local'},loglevel=0)#,'FILTER_NAME':'gauss_5.0_9x9.conv'},

diffim = './diff/F200W_epoch2_epoch1.diff.fits'
# temp = fits.open(diffim)
# temp_data = temp[0].data
# wcs = WCS(temp[0])

# fits.HDUList([fits.PrimaryHDU(temp_data)]).writeto('./diff/test.fits',overwrite=True)
class_star_cut = 0

# found = sew('./diff/test.fits')['table']
found = sew(diffim)['table']
# os.remove('./diff/test.fits')
found['xcentroid'] = found['X_IMAGE']-1
found['ycentroid'] = found['Y_IMAGE']-1
found['skycoord_peak'] = wcs.pixel_to_world(found['xcentroid'],found['ycentroid'])
found = found[found['CLASS_STAR']>class_star_cut]

table_cols = ['source_id','ra','dec']
table_data = [
    ['src_'+str(x) for x in np.arange(0,len(found),1)],
    [x.ra.value for x in found['skycoord_peak']],
    [x.dec.value for x in found['skycoord_peak']]
]

source_cat = Table(names=table_cols,dtype=[str]+[float]+[float])

for data_row in zip(*table_data):
    new_row = {}
    for i, d in enumerate(data_row):
        new_row[table_cols[i]] = d
    source_cat.add_row(new_row)

source_cat.write("./diff/test.txt",format='ascii',overwrite=True)

create_pixregionfile(source_cat['ra'], source_cat['dec'], './diff/test.reg', color="red",
                                 coords="fk5", radius=[0.4] * len(source_cat))

found

Ouch, SExtractor complains :
b''


X_IMAGE,Y_IMAGE,FLUX_RADIUS,FLUX_RADIUS_1,FLUX_RADIUS_2,FLAGS,XPEAK_WORLD,YPEAK_WORLD,CLASS_STAR
pix,pix,pix,pix,pix,,deg,deg,
float64,float64,float64,float64,float64,int64,float64,float64,float64
776.6957,9.3295,0.786,1.097,1.518,0,53.13826388,-27.825905347,0.422
794.3035,8.3438,1.266,1.861,3.086,0,53.138103691,-27.825913683,0.039
1786.8116,12.652,0.552,0.75,1.042,0,53.12874675,-27.825871848,0.133
766.2476,28.0074,1.198,1.735,2.994,0,53.138367528,-27.825747012,0.036
1812.2726,46.7807,-0.471,-0.785,-1.256,3,53.128511197,-27.825588502,0.075
753.329,34.0151,1.023,1.322,1.793,0,53.138490025,-27.82569701,0.053
1731.3068,31.6785,0.734,0.991,1.355,0,53.129274442,-27.825713541,0.64
1807.6554,44.7092,0.93,1.212,-0.662,3,53.12855831,-27.825605171,0.017


In [4]:
# mask_path = "./diff/F200W_epoch2_epoch1.diff.mask.fits"
# mask = fits.open(mask_path)[0].data.astype(bool)
# mask = fits.open(mask_path)[0].data != 0

diffim = './diff/F200W_epoch2_epoch1.diff.fits'
temp = fits.open(diffim)
temp_data = temp[0].data
wcs = WCS(temp[0])

In [5]:
detection_limit_flux = space_phot.cal.JWST_mag_to_flux(detection_limit,wcs)

In [6]:
print(detection_limit_flux)

[1.08294838]


In [8]:
fits.HDUList([fits.PrimaryHDU(temp_data)]).writeto('./diff/test.fits',overwrite=True)

class_star_cut = 0

found = sew('./diff/test.fits')['table']
os.remove('./diff/test.fits')
found['xcentroid'] = found['X_IMAGE']-1
found['ycentroid'] = found['Y_IMAGE']-1
found['skycoord_peak'] = wcs.pixel_to_world(found['xcentroid'],found['ycentroid'])
found = found[found['CLASS_STAR']>class_star_cut]



Ouch, SExtractor complains :
b''


In [9]:
found

X_IMAGE,Y_IMAGE,FLUX_RADIUS,FLUX_RADIUS_1,FLUX_RADIUS_2,FLAGS,XPEAK_WORLD,YPEAK_WORLD,CLASS_STAR,xcentroid,ycentroid,skycoord_peak
pix,pix,pix,pix,pix,,deg,deg,,pix,pix,"deg,deg"
float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,SkyCoord
776.6957,9.3295,0.786,1.097,1.518,0,776.0,8.0,0.422,775.6957,8.3295,"53.138266747376655,-27.82590260105035"
794.3035,8.3438,1.266,1.861,3.086,0,793.0,7.0,0.039,793.3035,7.3438,"53.138100830946755,-27.825910817914632"
1786.8116,12.652,0.552,0.75,1.042,0,1786.0,12.0,0.133,1785.8116,11.652,"53.12874852492021,-27.82587474763566"
766.2476,28.0074,1.198,1.735,2.994,0,765.0,27.0,0.036,765.2476,27.0074,"53.13836519531612,-27.825746950192556"
1812.2726,46.7807,-0.471,-0.785,-1.256,3,1811.0,46.0,0.075,1811.2726,45.7807,"53.12850862787806,-27.82559032919475"
753.329,34.0151,1.023,1.322,1.793,0,752.0,33.0,0.053,752.329,33.0151,"53.138486924678105,-27.825696883835178"
1731.3068,31.6785,0.734,0.991,1.355,0,1730.0,31.0,0.64,1730.3068,30.6785,"53.12927155085401,-27.8257162195386"
1807.6554,44.7092,0.93,1.212,-0.662,3,1806.0,44.0,0.017,1806.6554,43.7092,"53.12855213400839,-27.82560759401284"


In [10]:
found['skycoord_peak']

<SkyCoord (ICRS): (ra, dec) in deg
    [(53.13826675, -27.8259026 ), (53.13810083, -27.82591082),
     (53.12874852, -27.82587475), (53.1383652 , -27.82574695),
     (53.12850863, -27.82559033), (53.13848692, -27.82569688),
     (53.12927155, -27.82571622), (53.12855213, -27.82560759),
     (53.12848195, -27.82554014), (53.12866837, -27.82567456),
     (53.13824503, -27.82560953), (53.12856381, -27.82555753),
     (53.12844774, -27.82551259), (53.1284499 , -27.82556777),
     (53.1285044 , -27.82555993), (53.12909977, -27.8255339 ),
     (53.13852321, -27.82548864), (53.1387054 , -27.82524978),
     (53.13646166, -27.82524007), (53.13497356, -27.8250736 ),
     (53.13781335, -27.82497671), (53.13895499, -27.82496933),
     (53.13539915, -27.82492997), (53.13897386, -27.82489125),
     (53.13505995, -27.82480917), (53.12704085, -27.82471211),
     (53.12655901, -27.82476012), (53.13532182, -27.8247303 ),
     (53.13472291, -27.82468666), (53.13489005, -27.82455351),
     (53.13514935, -

In [11]:

table_cols = ['source_id','ra','dec']
table_data = [
    ['src_'+str(x) for x in np.arange(0,len(found),1)],
    [x.ra.value for x in found['skycoord_peak']],
    [x.dec.value for x in found['skycoord_peak']]
]

source_cat = Table(names=table_cols,dtype=[str]+[float]+[float])

for data_row in zip(*table_data):
    new_row = {}
    for i, d in enumerate(data_row):
        new_row[table_cols[i]] = d
    source_cat.add_row(new_row)

source_cat.write("./diff/test.txt",format='ascii',overwrite=True)

create_pixregionfile(source_cat['ra'], source_cat['dec'], './diff/test.reg', color="red",
                                 coords="fk5", radius=[0.4] * len(source_cat))

# Section 2: Differencing Lvl3 Mosaics Three Ways

### Inputs
* Ref and Sci epochs in the same tangential plane and same reference pixel

### Outputs
* Difference image (SCI and ERR arrays)

### Steps
1. Discuss general book keeping (file/folder conventions)
2. Perform a straight difference
3. Perform a difference using HOTPANTS
4. Perform a difference using SFFT

---

# Section 3: Source Extraction

### Inputs
* At least one difference image
* Input photo-z catalog (optional)

### Outputs
* ASCII-formatted file with source locations

### Steps
* Run photutils on difference image (DAOStarFinder)
* Match position to photo-z catalog 

<hr style="border:1px solid gray"> </hr>